In [29]:
%pip install sentence-transformers ollama

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Using cached ollama-0.5.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached ollama-0.5.1-py3-none-any.whl (13 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp311-c

In [3]:
import pandas as pd
from textwrap import dedent
# templates = pd.read_csv("templates_salmane/tenancy.csv")
# templates.head(2)

In [2]:
print("Shape : ", templates.shape
      , "\nColumns : ", templates.columns.tolist(),
)

Shape :  (1000, 3) 
Columns :  ['Template', 'prompt', 'contract']


In [3]:
# Create a new dataframe from scratch with specific columns
new_df = pd.DataFrame({
    "system": [],
    "user_prompt": [],
    "context": [],
    "reasoning_trace": [],
    "output": [],
    "metadata": []
})

In [4]:
new_df

,system,user_prompt,context,reasoning_trace,output,metadata


In [8]:
SYSTEM = dedent("""Vous êtes un assistant juridique spécialisé dans le droit marocain.
        Votre tâche est de générer des contrats complets et conformes aux lois marocaines.
        La génération doit etre en français, il sera bien de reflechir étape par étape (aussi en français si possible).
        Vous devez utiliser le contexte juridique fourni pour vous assurer que la clause est conforme aux lois marocaines.
        Vous devez également vous assurer que la clause répond aux besoins de l'entreprise.
        En ce qui concerne le format du contrat, vous devez suivre les conventions de rédaction des contrats au Maroc (Articles, Clauses, etc.).
        Vous DEVEZ vous baser EXCLUSIVEMENT sur le contexte juridique fourni.
        NE PAS inventer d'informations. Citez les articles de loi pertinents lorsque c'est possible.
        Soyez précis, formel et clair.""")

### Generate context for a given text

In [6]:
def retrieve_context(query, index, model, top_k=6):
    """Retrieves relevant legal context from Pinecone."""
    query_embedding = model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)
    
    # Format the retrieved data into a string context
    context = "CONTEXTE JURIDIQUE PERTINENT (Source: Code Marocain des Obligations et des Contrats):\n"
    for match in results['matches']:
        context += f"- Article {match['id']} ({match['metadata']['title']}): {match['metadata']['content']}\n"
    return context

In [7]:
PINECONE_API_KEY="pcsk_3GiCub_D2JbgoiCcpWyjRXKJ17QJ48Aq8E65oNiiwn3YT12RTJUCfKjsHd8jxxa9HmMCVu"

In [8]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import os

pc = Pinecone(api_key=PINECONE_API_KEY)
model = SentenceTransformer('intfloat/multilingual-e5-large')
# query_embedding = model.encode(templates.prompt[0]).tolist()

/home/gpuadmin/threathunting/phd/legal_ai/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
print(templates.prompt[0])


        Rédige un contrat de bail en français entre un bailleur et un locataire (Société). Le contrat doit inclure les informations suivantes :

            - Identité du bailleur : Mr REDA Ouazzani demeure à Fès titulaire de la carte CIN N° F393326

            - Identité du locataire (Société): Hobby Lobby sise à Laâyoune avec un capital 36M Dirhams representée par Mme hasna Sebti a pour activité: electric services

            - Adresse du bien loué : Laazib, Laâyoune, 36260, Morocco

            - Durée du bail : 1 année du 17-11-2010

            - Montant du loyer mensuel : 8000

        Utilise un ton juridique clair, structuré, et conforme à la législation locative en vigueur.
      


In [10]:
context = retrieve_context(templates.prompt[1], pc.Index("legalai"), model, top_k=6)
print("Context : ", context)

Context :  CONTEXTE JURIDIQUE PERTINENT (Source: Code Marocain des Obligations et des Contrats):
- Article 3 (Contenu du contrat de bail): Le contrat de bail doit être établi par acte écrit à date certaine et mentionner particulièrement : - Les nom et prénom du bailleur et du locataire, la profession, l'adresse, la pièce attestant de l'identité et toutes les informations relatives au mandataire, le cas échéant ; - La dénomination complète et le siège social et si nécessaire toutes les informations relatives au représentant légal si le bailleur ou le locataire est une personne morale ; - La désignation des locaux loués et de leurs dépendances, la destination ainsi que les équipements destinés à l'usage personnel du locataire ; - Le montant du loyer convenu et l'échéance de paiement ; - La nature des charges locatives qui seront supportées par le locataire ; - Le mode de paiement convenu du loyer et des charges locatives ; - Les obligations spécifiques de chaque partie.
- Article 39 (Int

In [12]:
testing_sample = templates.iloc[:50]

In [19]:
import tqdm

for i, row in tqdm.tqdm(templates.iterrows(), total=templates.shape[0]):
    user_prompt = row['prompt']
    context = retrieve_context(user_prompt, pc.Index("legalai"), model, top_k=6)
    
    # Generate the reasoning trace
    # reasoning_trace = f"Analyse de la demande : {user_prompt}\nContexte juridique pertinent : {context}\n"
    reasoning_trace = ""
    # Generate the output (contract clause)
    output = row['contract']
    
    # Create a new row as a DataFrame and concatenate it
    new_row = pd.DataFrame({
        "system": [SYSTEM],
        "user_prompt": [user_prompt],
        "context": [context],
        "reasoning_trace": [reasoning_trace],
        "output": [output],
        "metadata": [{"source": "templates_salmane/tenancy.csv", "index": i, "template_id" : row['Template'], "synthetic": False}]
    })
    
    new_df = pd.concat([new_df, new_row], ignore_index=True)

100%|██████████| 1000/1000 [13:18<00:00,  1.25it/s]


In [20]:
new_df

,system,user_prompt,context,reasoning_trace,output,metadata
0,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
2,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
3,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
4,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
...,...,...,...,...,...,...
1045,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1046,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1047,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1048,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."


In [22]:
new_df.drop_duplicates(subset=["output"], inplace=True)

In [4]:
new_df = pd.read_csv("templates_salmane/tenancy_processed.csv")
new_df

,system,user_prompt,context,reasoning_trace,output,metadata
0,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
2,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
3,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
4,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
...,...,...,...,...,...,...
995,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
996,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
997,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
998,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,NaN,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."


In [27]:
new_df.to_csv("templates_salmane/tenancy_processed.csv", index=False)

In [26]:
new_df

,system,user_prompt,context,reasoning_trace,output,metadata
0,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
2,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
3,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
4,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
...,...,...,...,...,...,...
1045,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1046,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1047,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1048,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,,CONTRAT DE BAIL A USAGE DE BUREAU\n\nENTRE-LES...,"{'source': 'templates_salmane/tenancy.csv', 'i..."


In [18]:
from ollama import Client

def get_llm_response(llm_pipeline: Client, prompt, model: str = None, mode = "generate"):
    """Gets a response from the LLM using the specified model."""
    if model is None:
        model = "deepseek-r1:32b"
        if model is None:
            raise ValueError("No model specified and OLLAMA_MODEL environment variable is not set")
    
    # print(f"Generating LLM response with model: {model}")
    if mode == "chat":
        sequences = llm_pipeline.chat(
            model=model,
            messages=prompt,
            keep_alive=100,
        )
        return sequences
    else:
        # Default to generate mode
        sequences = llm_pipeline.generate(
            model=model,
            prompt=prompt,
        )
    return sequences['response']

In [6]:
llm_pipeline = Client(host="http://localhost:11434")

In [9]:
generate_reasoning = f"""
    {SYSTEM}
    
    {new_df.iloc[0]['context']}
    {new_df.iloc[0]['user_prompt']}

    """

In [10]:
generate_reasoning

"\n    Vous êtes un assistant juridique spécialisé dans le droit marocain.\n        Votre tâche est de générer des contrats complets et conformes aux lois marocaines.\n        La génération doit etre en français, il sera bien de reflechir étape par étape (aussi en français si possible).\n        Vous devez utiliser le contexte juridique fourni pour vous assurer que la clause est conforme aux lois marocaines.\n        Vous devez également vous assurer que la clause répond aux besoins de l'entreprise.\n        En ce qui concerne le format du contrat, vous devez suivre les conventions de rédaction des contrats au Maroc (Articles, Clauses, etc.).\n        Vous DEVEZ vous baser EXCLUSIVEMENT sur le contexte juridique fourni.\n        NE PAS inventer d'informations. Citez les articles de loi pertinents lorsque c'est possible.\n        Soyez précis, formel et clair.\n\n    CONTEXTE JURIDIQUE PERTINENT (Source: Code Marocain des Obligations et des Contrats):\n- Article 3 (Contenu du contrat de

In [54]:
# get_llm_response(llm_pipeline, generate_reasoning, model="qwen3:8b")
get_llm_response(llm_pipeline, generate_reasoning, model="deepseek-r1:32b")

Generating LLM response with model: deepseek-r1:32b


"<think>\nOkay, I need to draft a lease agreement in French between a landlord and a tenant company based on the given context. Let me go through each part step by step.\n\nFirst, the contract should start with the identification of both parties. The landlord is Mr. Reda Ouazzani from Fès, with his CIN number provided. The tenant is Hobby Lobby, a company based in Laâyoune with a certain capital and represented by Mme Hasna Sebti. I need to make sure all their details are correctly stated.\n\nNext, the property address is given as Laazib, Laâyoune, 36260, Morocco. That needs to be clearly mentioned along with its usage, probably for commercial purposes since it's a company.\n\nThe lease duration is one year starting from November 17, 2010. I should specify the start and end dates in the contract. The monthly rent is 8000 Dirhams, which needs to be detailed regarding payment terms, like when and how it's paid.\n\nLooking at the legal context, Article 3 requires the contract to include b

In [ ]:
for i, row in tqdm.tqdm(new_df.iterrows(), total=new_df.shape[0]):
    user_prompt = row['user_prompt']
    context = row['context']
    
    generate_reasoning = f"""
    {SYSTEM}
    
    {context}
    {user_prompt}

    """

    response = get_llm_response(llm_pipeline, generate_reasoning, model="qwen3:8b")
    # Extract the reasoning trace from the response (limited by <think> and </think>  tags)
    reasoning_trace = response.split("<think>")[1].split("</think>")[0] if "<think>" in response else "No reasoning trace generated."
    print(f"Generated reasoning trace for row {i}: {reasoning_trace}")
    # Update the new_df with the generated reasoning trace
    new_df.at[i, 'reasoning_trace'] = reasoning_trace

  0%|          | 0/5 [00:00<?, ?it/s]

Generating LLM response with model: qwen3:8b


 20%|██        | 1/5 [00:32<02:10, 32.58s/it]

Generated reasoning trace for row 0: 
Okay, let me tackle this query step by step. The user wants a lease agreement in French between a landlord and a company, following Moroccan law. First, I need to recall the relevant articles from the Code Marocain des Obligations et des Contrats. 

The user provided specific details: the landlord is Mr. Reda Ouazzani with his CIN and address in Fès. The tenant is Hobby Lobby, a company in Laâyoune with a representative, Mme Hasna Sebti. The property is at Laazib, Laâyoune, with a specific postal code. The lease term is one year starting 17-11-2010, and the monthly rent is 8000 Dhs.

Starting with the structure, the lease must be in writing and dated, as per Article 3. I need to include all the required details: identities of both parties, their legal status, the property description, rent, charges, payment terms, and obligations. 

First, the parties' information. The landlord is a natural person, so I'll mention his full name, CIN, address, and p

 40%|████      | 2/5 [00:57<01:23, 27.82s/it]

Generated reasoning trace for row 1: 
Okay, I need to draft a lease contract in French between a landlord and a company tenant, following Moroccan law. Let me start by recalling the relevant articles from the Code Marocain des Obligations et des Contrats. 

First, Article 3 requires the lease to be in writing, including details about both parties. The landlord is Mr. MOHCINE Jamal with his CIN and address. The tenant is 9Yards Media, a company, so I need to include their legal name, address, capital, and representative. 

Article 39 talks about subletting, which is prohibited for residential use unless agreed in writing. Since the lease is for a commercial property (consumer services), maybe subletting isn't an issue here, but I should still mention it to be safe. 

Article 19 defines the tenant's responsibility for repairs, so I need to include clauses about maintenance and repairs. Article 10 says the landlord must maintain the property, but if they don't, the tenant can perform repa

 40%|████      | 2/5 [01:10<01:46, 35.34s/it]


KeyboardInterrupt: 

## Trying to parallelize stuff

In [ ]:
import threading
from concurrent.futures import ThreadPoolExecutor
import queue

# Create two LLM clients for different GPUs
llm_pipeline_gpu0 = Client(host="http://localhost:11434")
llm_pipeline_gpu1 = Client(host="http://localhost:11434")  # Assuming second Ollama instance on different port

# Or if using single Ollama instance, we'll use threading to manage requests
llm_pipelines = [llm_pipeline_gpu0, llm_pipeline_gpu1]

def process_row_batch(batch_data):
    """Process a batch of rows with a specific GPU pipeline"""
    pipeline_idx, rows_batch = batch_data
    pipeline = llm_pipelines[pipeline_idx % len(llm_pipelines)]
    
    results = []
    for i, row in rows_batch:
        user_prompt = row['user_prompt']
        context = row['context']
        
        generate_reasoning = f"""
        {SYSTEM}
        
        {context}
        {user_prompt}

        """

        response = get_llm_response(pipeline, generate_reasoning, model="qwen3:8b")
        reasoning_trace = response.split("<think>")[1].split("</think>")[0] if "<think>" in response else "No reasoning trace generated."
        
        results.append((i, reasoning_trace))
        print(f"GPU {pipeline_idx}: Generated reasoning trace for row {i}")
    
    return results

# Split the dataframe into batches for each GPU
batch_size = len(new_df) // 2
batches = [
    (0, list(new_df.iloc[:batch_size].iterrows())),
    (1, list(new_df.iloc[batch_size:].iterrows()))
]

# Process batches in parallel
with ThreadPoolExecutor(max_workers=2) as executor:
    futures = [executor.submit(process_row_batch, batch) for batch in batches]
    
    # Collect results
    for future in futures:
        results = future.result()
        for i, reasoning_trace in results:
            new_df.at[i, 'reasoning_trace'] = reasoning_trace

GPU 0: Generated reasoning trace for row 0
GPU 1: Generated reasoning trace for row 500
GPU 0: Generated reasoning trace for row 1
GPU 1: Generated reasoning trace for row 501
GPU 0: Generated reasoning trace for row 2
GPU 1: Generated reasoning trace for row 502
GPU 1: Generated reasoning trace for row 503
GPU 0: Generated reasoning trace for row 3
GPU 1: Generated reasoning trace for row 504
GPU 0: Generated reasoning trace for row 4
GPU 1: Generated reasoning trace for row 505
GPU 0: Generated reasoning trace for row 5
GPU 1: Generated reasoning trace for row 506
GPU 0: Generated reasoning trace for row 6
GPU 1: Generated reasoning trace for row 507
GPU 0: Generated reasoning trace for row 7
GPU 1: Generated reasoning trace for row 508
GPU 0: Generated reasoning trace for row 8
GPU 0: Generated reasoning trace for row 9
GPU 1: Generated reasoning trace for row 509


In [45]:
gen_reasoning_trace = f"""
You are a legal assistant specialized in Moroccan law.
Your task is to generate step-by-step legal thinking trace reasoning based on a user request, relevant legal context and the generated contract.
You will be provided with the user request, relevant legal context and the generated contract.
Your output should be an informal thinking trace reasoning that analyzes the user request, relevant legal context and the generated contract to generate a step-by-step legal reasoning. Like deepseek models do.
In other words, you should provide the thinking process that led to the generated contract, based on the user request and the relevant legal context provided.
Just like the example below, you should provide a detailed monologue that explains how you arrived at the generated contract, in your writing use (Je dois, Je vais, Je dois m'assurer, etc.) to reflect the legal reasoning process.
The output should be in French.

## Demande utilisateur :
{user_prompt}
## Contexte juridique pertinent :
{context}
## Contrat généré :
{output}
## Réflexion juridique étape par étape (MAKE SURE IT IS IN FRENCH):
D'accord, je dois 
"""

prompt = gen_reasoning_trace.format(
    user_prompt=new_df.iloc[0]['user_prompt'],
    context=new_df.iloc[0]['context'],
    output=new_df.iloc[0]['output']
)

In [47]:
print(prompt)


You are a legal assistant specialized in Moroccan law.
Your task is to generate step-by-step legal thinking trace reasoning based on a user request, relevant legal context and the generated contract.
You will be provided with the user request, relevant legal context and the generated contract.
Your output should be an informal thinking trace reasoning that analyzes the user request, relevant legal context and the generated contract to generate a step-by-step legal reasoning. Like deepseek models do.
In other words, you should provide the thinking process that led to the generated contract, based on the user request and the relevant legal context provided.
Just like the example below, you should provide a detailed monologue that explains how you arrived at the generated contract, in your writing use (Je dois, Je vais, Je dois m'assurer, etc.) to reflect the legal reasoning process.
The output should be in French.

## Demande utilisateur :

        Rédige un contrat de bail en français 

In [46]:
get_llm_response(llm_pipeline, prompt, model="deepseek-r1:32b")

Generating LLM response with model: deepseek-r1:32b


"<think>\nOkay, I need to figure out how to approach this legal reflection step by step in French for the provided lease agreement. Let me break it down.\n\nFirst, I'll read through the contract carefully to understand all the clauses and terms. It's important to grasp each section before analyzing them legally.\n\nNext, I should check if the lease meets essential legal requirements. For example, does it have a clear duration? Is there a termination clause? Are the parties properly identified?\n\nThen, I need to assess any potential risks or issues in the contract. Maybe some clauses are too one-sided towards the landlord. I should think about whether these terms are fair and if they comply with local laws.\n\nI'll also look at each article in detail. Article 6 talks about rent and payment deadlines. If the tenant misses a payment, the lease can be terminated after 15 days. That seems strict but might be standard. However, I should verify if this aligns with current legal standards.\n\

In [30]:
for i, row in tqdm.tqdm(new_df.iloc[:5].iterrows(), total=5):
    user_prompt = row['user_prompt']
    context = row['context']
    output = row['output']
    
    prompt = gen_reasoning_trace.format(
        user_prompt=user_prompt,
        context=context,
        output=output
    )
    
    reasoning_trace = get_llm_response(llm_pipeline, prompt, model="deepseek-r1:32b")
    
    # Update the new_df with the generated reasoning trace
    new_df.at[i, 'reasoning_trace'] = reasoning_trace

  0%|          | 0/5 [00:00<?, ?it/s]

Generating LLM response with model: deepseek-r1:32b


 20%|██        | 1/5 [01:07<04:29, 67.35s/it]

Generating LLM response with model: deepseek-r1:32b


 40%|████      | 2/5 [01:53<02:44, 54.88s/it]

...

 60%|██████    | 3/5 [03:32<02:29, 74.85s/it]

Generating LLM response with model: deepseek-r1:32b


 80%|████████  | 4/5 [04:45<01:14, 74.40s/it]

Generating LLM response with model: deepseek-r1:32b


100%|██████████| 5/5 [06:02<00:00, 72.50s/it]


In [32]:
new_df.reasoning_trace[1]

"<think>\nOkay, I need to help analyze this lease contract step by step. Let me start by reading through the contract thoroughly. It's in French, so I'll make sure I understand each clause properly.\n\nFirst, I notice that Article 5 talks about the monthly rent being 9000 DHS, with taxes paid by the tenant. The payment is due on the 5th of each month. If a single rent payment isn't made exactly on time, and after 15 days with no response to a formal notice, the lease can be terminated immediately.\n\nHmm, that seems pretty strict. I wonder if this aligns with local laws in France or wherever this is applicable. Maybe there are specific regulations about how much notice must be given before termination.\n\nNext, looking at Article 6, it mentions that any disputes will go to the courts in Rabat. I should check where the parties involved are located because court jurisdiction can sometimes depend on the location of the tenant or the property.\n\nIn Article 7, there's a provision for revis

In [2]:
import pandas as pd

df = pd.read_csv("templates_salmane/tenancy_processed_with_reasoning.csv")
df.head(2)

,system,user_prompt,context,reasoning_trace,output,metadata
0,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,"\nOkay, I need to create a lease agreement in ...",CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."
1,Vous êtes un assistant juridique spécialisé da...,\n Rédige un contrat de bail en françai...,CONTEXTE JURIDIQUE PERTINENT (Source: Code Mar...,"\nOkay, I need to create a lease contract in F...",CONTRAT DE BAIL\n\n\n\nEntre les soussignés:\n...,"{'source': 'templates_salmane/tenancy.csv', 'i..."


In [4]:
prompt = f"""
{df.iloc[0]['system']}\n

{df.iloc[0]['context']}
{df.iloc[0]['user_prompt']}

"""

In [5]:
prompt

"\nVous êtes un assistant juridique spécialisé dans le droit marocain.\n        Votre tâche est de générer des contrats complets et conformes aux lois marocaines.\n        La génération doit etre en français, il sera bien de reflechir étape par étape (aussi en français si possible).\n        Vous devez utiliser le contexte juridique fourni pour vous assurer que la clause est conforme aux lois marocaines.\n        Vous devez également vous assurer que la clause répond aux besoins de l'entreprise.\n        En ce qui concerne le format du contrat, vous devez suivre les conventions de rédaction des contrats au Maroc (Articles, Clauses, etc.).\n        Vous DEVEZ vous baser EXCLUSIVEMENT sur le contexte juridique fourni.\n        NE PAS inventer d'informations. Citez les articles de loi pertinents lorsque c'est possible.\n        Soyez précis, formel et clair.\n\n\nCONTEXTE JURIDIQUE PERTINENT (Source: Code Marocain des Obligations et des Contrats):\n- Article 3 (Contenu du contrat de bail)

In [21]:
llm_pipeline = Client(host="http://localhost:11434")

chat_template = [
  {
    'role': 'user',
    'content': prompt,
  },
]

response = get_llm_response(llm_pipeline, chat_template, model="legalai_qwen3_finetuned:latest", mode="chat")

In [27]:
response.message['content']

'<think>\nOkay, I need to create a lease agreement in French between a landlord and a tenant company. Let me start by recalling the relevant Moroccan laws. The user provided several articles from the Code Marocain des Obligations et des Contrats, so I should make sure the contract complies with those.\n\nFirst, the lease must be in writing, as per Article 3. I need to include the identities of both the landlord and the tenant. The landlord is Mr. Reda Ouazzani with a CIN number and address in Fès. The tenant is Hobby Lobby, a company based in Laâyoune with a capital of 36M DH, represented by Mme Hasna Sebti, in the electric services sector. \n\nNext, the leased property is at Laazib, Laâyoune, 36260, Morocco. The lease duration is one year starting November 17, 2010. The monthly rent is 8000 DH. \n\nI should structure the contract with the necessary sections: identification of parties, object of the lease, duration, rent, payment terms, obligations, termination, and other clauses. Also